In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [2]:
train = pd.read_csv("input/train.csv", index_col=0)
pred = pd.read_csv("input/predict.csv", index_col=0)

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.8" 2020-07-14 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.8+10-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.8+10-LTS, mixed mode)
  Starting server from /usr/local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/sk/1lhjy0853tx4hgjhly1z34pc0000gn/T/tmpwwqq3i9n
  JVM stdout: /var/folders/sk/1lhjy0853tx4hgjhly1z34pc0000gn/T/tmpwwqq3i9n/h2o_Marta_started_from_python.out
  JVM stderr: /var/folders/sk/1lhjy0853tx4hgjhly1z34pc0000gn/T/tmpwwqq3i9n/h2o_Marta_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,"7 days, 17 hours and 13 minutes"
H2O_cluster_name:,H2O_from_python_Marta_lkoymm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [4]:
h2train = h2o.H2OFrame(train)
h2pred = h2o.H2OFrame(pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
x = list(train.drop(columns='price').columns)
y = 'price'

In [52]:
automl = H2OAutoML(max_models=50, seed=1,max_runtime_secs=1200, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [53]:
leader_board = automl.leaderboard
leader_board.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_131327,546.81,299001,299001,274.511,0.0940663
StackedEnsemble_BestOfFamily_AutoML_20201005_131327,562.304,316185,316185,280.244,0.0960789
XGBoost_grid__1_AutoML_20201005_131327_model_1,566.624,321063,321063,282.947,0.0959238
XGBoost_1_AutoML_20201005_131327,578.533,334700,334700,293.591,nan
XGBoost_3_AutoML_20201005_131327,578.705,334900,334900,310.429,0.120563
XGBoost_2_AutoML_20201005_131327,593.136,351810,351810,298.424,0.0989703
GBM_grid__1_AutoML_20201005_131327_model_1,635.207,403487,403487,344.205,nan
DRF_1_AutoML_20201005_131327,722.767,522392,522392,358.56,0.122102
XRT_1_AutoML_20201005_131327,768.328,590327,590327,389.098,0.13807
GBM_1_AutoML_20201005_131327,935.334,874850,874850,578.017,0.320378


In [54]:
automl = H2OAutoML(max_models=50, seed=1,max_runtime_secs=1800, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [55]:
leader_board2 = automl.leaderboard
leader_board2.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_133736,542.346,294139,294139,274.5,0.0954452
StackedEnsemble_BestOfFamily_AutoML_20201005_133736,552.881,305677,305677,276.184,0.0958583
XGBoost_grid__1_AutoML_20201005_133736_model_1,562.98,316946,316946,281.199,0.0951049
XGBoost_3_AutoML_20201005_133736,568.504,323196,323196,303.591,0.117183
XGBoost_1_AutoML_20201005_133736,579.624,335964,335964,293.595,0.103092
XGBoost_2_AutoML_20201005_133736,587.683,345371,345371,295.431,nan
GBM_2_AutoML_20201005_133736,594.745,353721,353721,316.124,nan
XGBoost_grid__1_AutoML_20201005_133736_model_2,602.517,363026,363026,296.035,0.100667
GBM_grid__1_AutoML_20201005_133736_model_1,605.02,366050,366050,313.5,nan
GBM_3_AutoML_20201005_133736,611.209,373577,373577,345.238,nan


In [56]:
predictions = automl.leader.predict(h2pred)
predictions.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
750.394
6105.41
5882.33
1075.32
3130.97
538.64
1417.18
846.757
8893.59
678.95


In [63]:
sub = predictions.as_data_frame()

In [65]:
columns=['id','price']
sub = sub.reset_index(level=0)
sub.columns = columns

In [66]:
sub.to_csv("subm_6.csv",index=False)

In [67]:
automl = H2OAutoML(max_models=50,max_runtime_secs=3600, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [68]:
leader_board3 = automl.leaderboard
leader_board3.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_153521,541.402,293116,293116,274.192,0.097376
StackedEnsemble_BestOfFamily_AutoML_20201005_153521,544.166,296116,296116,276.387,nan
GBM_3_AutoML_20201005_153521,553.005,305814,305814,284.065,nan
GBM_1_AutoML_20201005_153521,555.29,308347,308347,290.09,0.106969
GBM_grid__1_AutoML_20201005_153521_model_5,557.775,311113,311113,278.705,0.0967335
GBM_2_AutoML_20201005_153521,558.373,311780,311780,289.586,nan
GBM_4_AutoML_20201005_153521,562.082,315936,315936,289.63,nan
GBM_grid__1_AutoML_20201005_153521_model_11,564.127,318240,318240,280.993,0.096471
XGBoost_grid__1_AutoML_20201005_153521_model_1,565.335,319603,319603,288.166,0.0999397
GBM_grid__1_AutoML_20201005_153521_model_2,566.973,321458,321458,283.905,nan


In [7]:
automl = H2OAutoML(max_runtime_secs=1000, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
leader_board3 = automl.leaderboard
leader_board3.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201005_185643,533.132,284229,284229,269.081,0.0953999
GBM_3_AutoML_20201005_185643,539.697,291273,291273,276.045,0.0996463
StackedEnsemble_BestOfFamily_AutoML_20201005_185643,540.264,291885,291885,273.347,0.0977598
GBM_2_AutoML_20201005_185643,543.333,295211,295211,279.838,0.103419
GBM_4_AutoML_20201005_185643,544.718,296718,296718,279.588,0.102665
GBM_1_AutoML_20201005_185643,552.806,305595,305595,289.88,0.107759
GBM_grid__1_AutoML_20201005_185643_model_3,562.803,316747,316747,297.508,nan
XGBoost_grid__1_AutoML_20201005_185643_model_2,584.91,342120,342120,301.209,nan
XGBoost_2_AutoML_20201005_185643,586.535,344023,344023,295.152,0.098874
XGBoost_1_AutoML_20201005_185643,589.787,347848,347848,303.323,nan


In [9]:
predictions = automl.leader.predict(h2pred)
sub = predictions.as_data_frame()
columns=['id','price']
sub = sub.reset_index(level=0)
sub.columns = columns
sub.to_csv("subm_8.csv",index=False)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [6]:
automl = H2OAutoML(max_runtime_secs=1000, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)
leader_board3 = automl.leaderboard
leader_board3.head()

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_004457,531.343,282325,282325,269.631,0.0961785
StackedEnsemble_BestOfFamily_AutoML_20201006_004457,535.75,287028,287028,270.913,0.0962415
GBM_2_AutoML_20201006_004457,539.058,290584,290584,277.665,0.102363
GBM_3_AutoML_20201006_004457,539.092,290620,290620,275.024,0.097921
GBM_4_AutoML_20201006_004457,539.143,290675,290675,271.332,nan
GBM_1_AutoML_20201006_004457,542.186,293966,293966,281.676,0.103869
GBM_grid__1_AutoML_20201006_004457_model_7,545.28,297330,297330,284.453,0.104547
GBM_grid__1_AutoML_20201006_004457_model_9,550.75,303325,303325,274.569,0.0909164
GBM_grid__1_AutoML_20201006_004457_model_4,551.368,304006,304006,274.062,0.0914225
GBM_grid__1_AutoML_20201006_004457_model_10,552.561,305324,305324,281.113,0.102103


In [7]:
predictions = automl.leader.predict(h2pred)
sub = predictions.as_data_frame()
columns=['id','price']
sub = sub.reset_index(level=0)
sub.columns = columns
sub.to_csv("subm_9.csv",index=False)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [6]:
automl = H2OAutoML(max_runtime_secs=1200, sort_metric='RMSE')
automl.train(x, y, training_frame=h2train)
leader_board3 = automl.leaderboard
leader_board3.head()

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_084246,529.277,280134,280134,267.294,nan
StackedEnsemble_BestOfFamily_AutoML_20201006_084246,535.271,286515,286515,270.885,nan
GBM_3_AutoML_20201006_084246,536.098,287401,287401,274.59,nan
GBM_4_AutoML_20201006_084246,537.858,289291,289291,270.091,nan
GBM_2_AutoML_20201006_084246,538.716,290215,290215,277.15,nan
GBM_1_AutoML_20201006_084246,542.538,294348,294348,281.991,0.103751
GBM_grid__1_AutoML_20201006_084246_model_5,543.682,295590,295590,277.136,0.101213
GBM_grid__1_AutoML_20201006_084246_model_6,545.023,297050,297050,275.492,0.0955505
GBM_grid__1_AutoML_20201006_084246_model_3,547.31,299548,299548,273.186,0.0961178
GBM_grid__1_AutoML_20201006_084246_model_4,549.073,301481,301481,271.664,0.0895764


In [7]:
predictions = automl.leader.predict(h2pred)
sub = predictions.as_data_frame()
columns=['id','price']
sub = sub.reset_index(level=0)
sub.columns = columns
sub.to_csv("subm_10.csv",index=False)

stackedensemble prediction progress: |████████████████████████████████████| 100%
